In [85]:
from ansys.mapdl.core import launch_mapdl
import pandas as pd

def launch_mapdl_on_available_port(starting_port=50052, max_attempts=10):
    for i in range(max_attempts):
        port = starting_port + i
        try:
            mapdl = launch_mapdl(port=port)
            print(f"MAPDL launched successfully on port {port}")
            return mapdl
        except Exception as e:
            print(f"Failed to launch MAPDL on port {port}: {e}")
    raise RuntimeError("Could not launch MAPDL on any available port")

# Use a função para iniciar uma instância do MAPDL
mapdl = launch_mapdl_on_available_port()

MAPDL launched successfully on port 50052


In [86]:
mapdl.clear('NOSTART')
mapdl.prep7()

# Título
mapdl.title('Análise de Estacas e Casca')

# Definir o tipo de elemento (BEAM188) e suas propriedades
mapdl.et(1, 'BEAM188')

# Propriedades do material
modulo_elasticidade = 0.85 * 5600 * (40 ** 0.5) * 1e6  # N/m²
mapdl.mp('EX', 1, modulo_elasticidade)
mapdl.mp('PRXY', 1, 0.2)  # Coeficiente de Poisson
mapdl.mp('DENS', 1, 2500)  # Densidade

# Propriedades da seção da viga
mapdl.sectype(1, 'BEAM', 'CSOLID')
mapdl.secoffset('CENT')
mapdl.secdata(0.5)

# Definir os pontos iniciais e finais das estacas
x_iniciais = [0.5, 2.0, 4.0, 5.5, 0.5, 5.5, 0.5, 2.0, 4.0, 5.5]
y_iniciais = [2.5, 2.5, 2.5, 2.5, 0.0, 0.0, -2.5, -2.5, -2.5, -2.5]
z_iniciais = [0.0] * 10
x_finais = [0.5, 2.0, 4.0, 5.5, 0.5, 5.5, 0.5, 2.0, 4.0, 5.5]
y_finais = [2.5, 2.5, 2.5, 2.5, 0.0, 0.0, -2.5, -2.5, -2.5, -2.5]
z_finais = [-20.0] * 10

# Número de nós intermediários
num_intermediate_nodes = 9

# Adicionar nós
node_id = 1
for i in range(10):
    x_inicial = x_iniciais[i]
    y_inicial = y_iniciais[i]
    z_inicial = z_iniciais[i]
    
    x_final = x_finais[i]
    y_final = y_finais[i]
    z_final = z_finais[i]
    
    mapdl.n(node_id, x_inicial, y_inicial, z_inicial)
    node_id_final = node_id + num_intermediate_nodes + 1
    mapdl.n(node_id_final, x_final, y_final, z_final)
    
    # Travar nós finais
    mapdl.d(node_id_final, 'ALL', 0)
    
    # Preencher nós intermediários
    mapdl.fill(node_id, node_id_final, num_intermediate_nodes)
    
    node_id = node_id_final + 1

# Geração de elementos
n_elemento = 1
for i in range(10):
    for j in range(1, num_intermediate_nodes + 2):
        N_1 = j + (num_intermediate_nodes + 2) * (i)
        N_2 = N_1 + 1
        mapdl.en(n_elemento, N_1, N_2)
        n_elemento += 1

# Selecionar elementos tipo BEAM188
mapdl.esel('S', 'TYPE', '', 1)

# Contar elementos selecionados
num_elem = mapdl.get('num_elem', 'ELEM', 0, 'COUNT')
k = num_elem

# Criar elemento de carga
mapdl.n(1000, 3, 0, 0)
mapdl.n(1001, 3, 0, 0.1)
mapdl.en(k + 1, 1000, 1001)

# Definir tipo de elemento SHELL181
mapdl.et(2, 'SHELL181')
mapdl.keyopt(2, 8, 2)  # Elastoplástico
mapdl.keyopt(2, 3, 2)  # Precisão de tensões

# Propriedades do material para SHELL181
modulo_elasticidade_shell = 0.85 * 5600 * (20 ** 0.5) * 1e6
mapdl.mp('EX', 2, modulo_elasticidade_shell)
mapdl.mp('PRXY', 2, 0.2)
mapdl.mp('DENS', 2, 2500)

# Definir seção de casca
mapdl.sectype(2, 'SHELL')
mapdl.secdata(1.5)

# Criar retângulo e malhar
mapdl.rectng(0, 6, -3, 3)
mapdl.esize(0.1)
mapdl.amesh('ALL')

# Selecionar elementos tipo SHELL181
mapdl.esel('S', 'TYPE', '', 2)
mapdl.emodif('ALL', 'SECNUM', 2)

# Merge de nós
mapdl.nsel('S', 'LOC', 'Z', 0, 1e5)
mapdl.nummrg('NODE', 1e-5)
mapdl.nsel('ALL')

# Saída dos resultados
mapdl.allsel('ALL')
mapdl.nlist('ALL', '', '', 'XYZ', 'NODE', 'NODE', 'NODE')
mapdl.elist('ALL', '', '', 0, 0)

# Aplicar gravidade
mapdl.acel(0, 0, -9.81)

# Finalizar
mapdl.finish()

***** ROUTINE COMPLETED *****  CP =         0.156

In [87]:
# Entrar no modo de solução
mapdl.slashsolu()
mapdl.antype(0)

PERFORM A STATIC ANALYSIS
  THIS WILL BE A NEW ANALYSIS

In [88]:
import math
# Definir a força aplicada e o número de load steps
f = 1000000  # Força de 1 milhão de N

# Loop para aplicar as diferentes condições de carga
for i in range(1, 4):
    # Calcular o ângulo em radianos
    Rad = (i-1)*45*math.pi/180

    # Calcular as componentes da força horizontal
    FX = math.cos(Rad)*f
    FY = math.sin(Rad)*f

    print(FX)
    print(FY)

    # Selecionar todos os elementos
    mapdl.allsel('ALL')

    # Aplicar as forças horizontais no nó 1001
    mapdl.f(1001, 'FX', FX)
    mapdl.f(1001, 'FY', FY)

    # Resolver o modelo
    mapdl.solve()

    # Salvar os resultados da solução
    mapdl.save(f'load_step_{i}')

# Finalizar
mapdl.finish()

1000000.0
0.0
707106.7811865476
707106.7811865476
6.123233995736767e-11
1000000.0


FINISH SOLUTION PROCESSING


 ***** ROUTINE COMPLETED *****  CP =         2.172

In [89]:
# Entrar no modo de pós-processamento
mapdl.post1()


# Definir os Load Cases para N = 3
for i in range(1, 4):
    mapdl.lcdef(i, i, 1)  # Define o Load Case i para o Load Step i 

# Carregar o primeiro Load Case
mapdl.lcase(1)

# Comparar o Load Case 1 com os demais e armazenar os resultados (máximos)
for R in range(2, 4): 
    mapdl.lcoper('MAX', R)  # Compara o Load Case na memória com os próximos
    mapdl.lcwrite(50 + R)   # Escreve o resultado em um arquivo

# Carregar o resultado do último load case comparado
mapdl.lcase(53)

# Gerar ETABLE para momentos fletores My e Mz dos elementos selecionados (valores máximos)
mapdl.etable('Fx_MAX', 'SMISC', 1, 'MAX')  # Força Axial máxima
mapdl.etable('My_MAX', 'SMISC', 2, 'MAX')  # Momento fletor máximo em Y
mapdl.etable('Mz_MAX', 'SMISC', 3, 'MAX')  # Momento fletor máximo em Z

# Carregar o primeiro Load Case novamente
mapdl.lcase(1)

# Comparar o Load Case 1 com os demais para valores mínimos
for R in range(2, 4):
    mapdl.lcoper('MIN', R)  # Compara o Load Case na memória para valores mínimos
    mapdl.lcwrite(60 + R)   # Escreve o resultado em um arquivo

# Carregar o resultado do último load case comparado
mapdl.lcase(63)

# Gerar ETABLE para momentos fletores e força axial (valores mínimos)
mapdl.etable('Fx_MIN', 'SMISC', 1, 'MIN')  # Força Axial mínima
mapdl.etable('My_MIN', 'SMISC', 2, 'MIN')  # Momento fletor mínimo em Y
mapdl.etable('Mz_MIN', 'SMISC', 3, 'MIN')  # Momento fletor mínimo em Z
mapdl.etable('Area', 'SMISC', 13)  # Área da seção transversal


STORE AREA     FROM ITEM=SMIS COMP=  13  FOR ALL SELECTED ELEMENTS

In [90]:
import pandas as pd

elementos_sequencia = []
j=1
for i in range(0,10, 1):
    y=39*i + j
    elementos_sequencia.append(y)
    elementos_sequencia.append(y+num_intermediate_nodes)         # Adiciona o primeiro elemento do intervalo
    j +=1

# Dicionário para armazenar os resultados
data = {
    "Element": [],
    "Fx_MAX": [],
    "Fx_MIN": [],
    "My_MAX": [],
    "My_MIN": [],
    "Mz_MAX": [],
    "Mz_MIN": [],
}

# Extraindo valores com mapdl.get_value para cada elemento na sequência
for elem in elementos_sequencia:
    fx_max = -1*mapdl.get_value('ELEM', elem, 'ETABLE', 'Fx_MAX')
    fx_min = -1*mapdl.get_value('ELEM', elem, 'ETABLE', 'Fx_MIN')
    my_max = mapdl.get_value('ELEM', elem, 'ETABLE', 'My_MAX')
    my_min = mapdl.get_value('ELEM', elem, 'ETABLE', 'My_MIN')
    mz_max = mapdl.get_value('ELEM', elem, 'ETABLE', 'Mz_MAX')
    mz_min = mapdl.get_value('ELEM', elem, 'ETABLE', 'Mz_MIN')

    # Adicionando os resultados à tabela
    data["Element"].append(elem)
    data["Fx_MAX"].append(fx_max)
    data["Fx_MIN"].append(fx_min)
    data["My_MAX"].append(my_max)
    data["My_MIN"].append(my_min)
    data["Mz_MAX"].append(mz_max)
    data["Mz_MIN"].append(mz_min)
# Criando o DataFrame com os resultados
df = pd.DataFrame(data)
df=df.round(0)
# Exibindo a tabela
print(df)

# Opcional: Salvar a tabela como arquivo CSV
#df.to_csv('element_results.csv', index=False)


    Element     Fx_MAX    Fx_MIN    My_MAX    My_MIN    Mz_MAX    Mz_MIN
0         1  -742963.0  346512.0  867844.0    -764.0    5760.0 -868080.0
1        10 -1089407.0      68.0     393.0 -924455.0  921186.0   -2702.0
2        41  -765601.0 -164603.0  873168.0    1452.0       0.0 -887732.0
3        50 -1112045.0 -511047.0    -669.0 -926997.0  930250.0      -0.0
4        81  -648166.0   55919.0  879543.0   -1384.0    -472.0 -876491.0
5        90  -994610.0 -290525.0     662.0 -929856.0  925117.0     201.0
6       121   -31738.0   55481.0    -485.0   -3594.0    4560.0     724.0
7       130    -7944.0  195141.0     666.0    -178.0    3431.0    1971.0
8       161    -2302.0    2550.0    2470.0   -2381.0    2884.0   -2981.0
9       170   -54158.0   27260.0    5274.0   -1250.0   31084.0   -5286.0
10      201    27094.0   73890.0   -2378.0  -15807.0    6993.0  -19527.0
11      210    47235.0   75699.0    4269.0    1897.0    4517.0  -14613.0
12      241   -23710.0   65944.0   -5259.0  -39485.

In [91]:
#Sair da port do mapdl e fechar o processo
mapdl.exit()
